In [1]:
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import datetime as dt

In [2]:
vendorsDF = pd.read_csv('./../data/vendorsCSVs/allVendors.csv')

In [3]:
grams_df = pd.read_csv('../saved_gen_df/agora_grams_df2.csv')

In [7]:
vendorsDF

,Unnamed: 0,Unnamed: 0.1,name,rating,lastSeen,publicKey,#products,#Deals,parsingDate
0,0,0,littlegoblin,4.92,4 hours,NaN,14.0,150~200,2014-09-20
1,1,1,Rebel_Alliance,4.99,1 hours,NaN,7.0,300~500,2014-09-20
2,2,2,ThePartyDepot,5.00,16 hours,NaN,3.0,70~100,2014-09-20
3,3,3,cossacks,4.99,5 hours,NaN,7.0,300~500,2014-09-20
4,4,4,checkpoint,4.84,1 days 0 hours,NaN,31.0,55~70,2014-09-20
...,...,...,...,...,...,...,...,...,...
121153,121153,404,BabyBando,5.00,Registered 7 months 8 days 3 hours,NaN,6.0,10~15,2014-10-14
121154,121154,405,daydreamer33,4.94,Registered 6 months 21 days 1 day...,NaN,6.0,200~300,2014-10-14
121155,121155,406,Leckermann,5.00,Registered 6 months 22 days 5 mon...,NaN,8.0,3~5,2014-10-14
121156,121156,407,EasyRiders,NaN,Registered 1 months 24 days 20 days,NaN,5.0,0,2014-10-14


In [11]:
temp = grams_df[['vendor_name', 'ship_from']].reset_index()
def fillCountry(n):
    rows= temp[temp.vendor_name == n]
    if(len(rows) > 0):
        return rows.iloc [0].ship_from
    else:
        return None

In [ ]:
vendorsDF['country'] = vendorsDF['name'].apply(fillCountry)

In [ ]:
vendorsDF

In [ ]:
def cleanLastSeenRegistered(s):
    s = str(s)
    if ('Registered' in s):
        return s.split('  ')[-1]
    else:
        return s

In [ ]:
vendorsDF['lastSeen'] = vendorsDF['lastSeen'].apply(cleanLastSeenRegistered)

In [ ]:
vendorsDF['parsingDate']  = vendorsDF['parsingDate'].apply((lambda x: dt.datetime.strptime(x, '%Y-%m-%d').date()))

In [ ]:
vendorsDF = vendorsDF.drop('Unnamed: 0', axis=1)

In [ ]:
vendorsDF.lastSeen = vendorsDF.lastSeen.apply((lambda x:addZeroPadding(x)))
vendorsDF.lastSeen = vendorsDF.lastSeen.apply((lambda x:cleanLastSeen(x)))

In [ ]:
vendorsDF

In [ ]:
def addZeroPadding(s):
    s ='a '+str(s)
    i =1
    while(i<10):
        looked =str(i)
        s= s.replace('a '+looked+' days','0'+looked+' days')
        s= s.replace('a '+looked+' hours','0'+looked+' hours')
        s= s.replace('a '+looked+' months','0'+looked+' months')
        s= s.replace(' '+looked+' days',' 0'+looked+' days')
        s= s.replace(' '+looked+' hours',' 0'+looked+' hours')
        s= s.replace(' '+looked+' months',' 0'+looked+' months')
        i+=1
    s = s.replace("a ",'')
    return s

In [ ]:
def cleanLastSeen(s):

    if ( 'days 0 hours' in s):
        t = dt.datetime.strptime(s,"%d days 0 hours")
        return dt.timedelta(days=t.day)
    elif ( 'months 0 days' in s):
        t = dt.datetime.strptime(s,"%m months 0 days")
        return dt.timedelta(days=30*t.month,hours=0)
    elif ( 'days' in s and 'hours' in s):
        t = dt.datetime.strptime(s,"$d days %H hours")
        return dt.timedelta(days=t.day,hours=t.hour)
    elif ( 'months' in s and 'days' in s):
        t = dt.datetime.strptime(s,"%m months %d days")
        return dt.timedelta(days=t.day+30*t.month,hours=0)
    elif ( 'hours' in s):
        t = dt.datetime.strptime(s,"%H hours")
        return dt.timedelta(hours=t.hour)
    else:
        return s;